In [3]:
#Author: Qian Wang
import pandas as pd
import numpy as np
import yelpapi
import rauth
import time
import json
import io
import codecs
import requests
from bs4 import BeautifulSoup as bs
from urllib import urlopen
import nltk
from nltk.corpus import stopwords
from nltk.stem import *
import re

import string
import sys  

#reload(sys)  
#sys.setdefaultencoding('utf8')

last_fetched_at = None

In [4]:
class Review_Extraction():
    def __init__(self, location=(40.7311019, -74.00137210000003), limit=20, radius_filter=1000,count=0,index=0):

        self.location = location
        self.limit = limit
        self.radius_filter = radius_filter
        self.index = index
        self.count = count
    def get_search_parameters(self, latitude, longitude, radius_filter, limit):
        # See the Yelp API for more details
        params = {}
        params["term"] = "restaurant"
        params["ll"] = "{},{}".format(str(latitude), str(longitude))
        params["radius_filter"] = str(radius_filter)
        params["limit"] = str(limit)

        return params

    def get_results(self, params):

        # Obtain these from Yelp's manage access page
        consumer_key = "qvtAmgqN2IgX1-x54gTQZw"
        consumer_secret = "_9YP_S2lsrT0kcVzGVKc--SM7Hw"
        token = "4dl-TfK8eitmcztp_H8CMznEb2z_r6w7"
        token_secret = "mC1jEXAi8jlP3lzsRIWgBcR6J3o"

        session = rauth.OAuth1Session(
            consumer_key=consumer_key
            , consumer_secret=consumer_secret
            , access_token=token
            , access_token_secret=token_secret)

        request = session.get("http://api.yelp.com/v2/search", params=params)

        # Transforms the JSON API response into a Python dictionary
        data = request.json()
        session.close()

        return data

    def findRestaurant(self):

        lati, longi = self.location
        params = self.get_search_parameters(lati, longi, self.radius_filter, self.limit)
        result = self.get_results(params)
        time.sleep(1.0)

        with io.open('restaurant_data_big'+str(self.index)+'.txt', 'w', encoding='utf-8') as f:
            f.write(unicode(json.dumps(result, ensure_ascii=False)))
        f.close()

        totalBusiness = result["total"]
        BizList = result["businesses"]

        columns = ["name", "is_claimed", "review_count", "url", "rating", "city"]
        data_df = pd.DataFrame(columns=columns)
        i = 0
        for each in BizList:
            isClaim = each["is_claimed"]
            rating = each["rating"]
            name = each["name"]
            url = each["url"]
            revCnt = each["review_count"]
            city = each["location"]["city"]
            row = [name, isClaim, revCnt, url, rating, city]
            data_df.loc[i] = row
            i += 1

        return data_df, totalBusiness, i

    def get_single_review(self, url):
        global last_fetched_at

        wait_interval = 2000
        if last_fetched_at is not None:
            now = time.time()
            elapsed = now - last_fetched_at
            if elapsed < wait_interval:
                time.sleep((wait_interval - elapsed) / 1000)

        page = urlopen(url)
        soup = bs(page, "lxml")
        reviews = soup.findAll('p', attrs={'itemprop': 'description'})
        authors = soup.findAll('span', attrs={'itemprop': 'author'})
        flag = True
        indexOf = 1
        content = ""
        nlp_content = ""
        for review in reviews:
            dirtyEntry = str(review)
            while dirtyEntry.index('<') != -1:
                indexOf = dirtyEntry.index('<')
                endOf = dirtyEntry.index('>')
                if flag:
                    dirtyEntry = dirtyEntry[endOf + 1:]
                    flag = False
                else:
                    if (endOf + 1 == len(dirtyEntry)):
                        cleanEntry = dirtyEntry[0:indexOf]
                        break
                    else:
                        dirtyEntry = dirtyEntry[0:indexOf] + dirtyEntry[endOf + 1:]

            content += cleanEntry

            eachEntry = cleanEntry.decode(encoding='UTF-8', errors='ignore')
            lowers_entry = eachEntry.lower()

            no_punctuation_entry = re.sub(r'[^\w\s]', '', lowers_entry)
            tokens_entry = nltk.word_tokenize(no_punctuation_entry)
            filtered_entry = [w for w in tokens_entry if not w in stopwords.words('english')]
            stemmer = PorterStemmer()
            for item in filtered_entry:
                nlp_content += (stemmer.stem(item) + " ")

        last_fetched_at = time.time()

        return content, nlp_content

    def get_reviews(self, url_list, filename1="review_whole_big", filename2="review_clean_big"):

        rec_len = len(url_list)
        with open(filename1+str(self.index)+".txt", 'w') as f:
            # for each in url_list:
            # print str(i) + " " + self.get_single_review(each)
            f.writelines((str(i+self.count) + " " + self.get_single_review(url_list[i])[0] + "\n") for i in range(rec_len - 1))
            f.writelines((str(self.count+rec_len - 1) + " " + self.get_single_review(url_list[rec_len - 1])[0]))

        f.close()

        with open(filename2+str(self.index)+".txt", 'w') as f:
            # for each in url_list:
            # print str(i) + " " + self.get_single_review(each)
            f.writelines((str(i+self.count) + " " + self.get_single_review(url_list[i])[1] + "\n") for i in range(rec_len - 1))
            f.writelines((str(self.count+rec_len - 1) + " " + self.get_single_review(url_list[rec_len - 1])[1]))

        f.close()


In [7]:
if __name__ == '__main__':
    locations = [(40.7311019, -74.00137210000003), (40.7167219, -73.99814459999999), (40.758895, -73.98513100000002), (40.706441, -74.0052976), (40.7794366, -73.96324400000003), (41.8773933, -87.6256477), (41.8942258,-87.61980819999997), (41.92979379999999, -87.65516830000001), (41.79611199999999, -87.58862899999997), (41.8728752, -87.65401689999999)]
    index = 0
    count = 0
    i = 0
    for location in locations:
        rev_ext = Review_Extraction(location,index=index,count=count)
        dftmp, numBiztmp, i = rev_ext.findRestaurant()
        print numBiztmp
        dftmp.to_csv("data_feature_big"+str(index)+".csv", encoding='utf8')
        rev_ext.get_reviews(dftmp['url'])
        count = count+i
        index = index+1
        time.sleep(1.0)


832
693
1231
513
113
355
278
107
58
52
